In [1]:
import re

# Corpus Read
with open('corpus_non_split.txt', 'r', encoding='utf-8') as f:
    corpus = f.read()

# Corpus 정제
repr_pattern  = '[^a-zA-Z가-힣\d\n]'
re_corpus = re.sub(pattern=repr_pattern, repl=' ', string=corpus)
re_corpus = re.sub(pattern='\s{2,}', repl=' ', string=re_corpus)

# Corpus 저장
with open('re_corpus.txt', 'w', encoding='utf-8') as f:
    f.write(re_corpus)

In [3]:
from tokenizers import SentencePieceBPETokenizer

s_tokenizer = SentencePieceBPETokenizer()

data_file = 're_corpus.txt'
vocab_size = 30000
limit_alphabet = 6000
min_frequency = 3

s_tokenizer.train(files=data_file,
                  vocab_size=vocab_size,
                  limit_alphabet=limit_alphabet,
                  min_frequency=min_frequency)



In [6]:
import pandas as pd
# 결과 확인
encoded = s_tokenizer.encode("행복주택에서 미소금융이나 햇살론을 농협은행에서 담보대출을")
print('토큰화 결과 :',encoded.tokens)
print('정수 인코딩 :',encoded.ids)
print('디코딩 :',s_tokenizer.decode(encoded.ids))

# 결과 저장
s_tokenizer.save_model('./tokenizer/')
# >>> ['./tokenizer/vocab.json', './tokenizer/merges.txt']

# vocabulary 확인
added_vocab = pd.read_json('./tokenizer/vocab.json', orient='index')
print("추가된 단어 수:", len(added_vocab))

토큰화 결과 : ['▁행복주택', '에서', '▁미소금융', '이나', '▁햇살론', '을', '▁농협', '은행', '에서', '▁담보', '대출', '을']
정수 인코딩 : [3744, 849, 5696, 1777, 2233, 526, 2403, 1142, 849, 3684, 1401, 526]
디코딩 : 행복주택에서 미소금융이나 햇살론을 농협은행에서 담보대출을
추가된 단어 수: 7253


In [8]:
from sentence_transformers import SentenceTransformer

# 모델 불러오기

base_model_path = "BM-K/KoSimCSE-roberta-multitask"
# 내가 사용하는 모델이다. 자신이 사용할 (허깅페이스의) 모델 이름 혹은 그 경로를 입력한다.
# 실행만 해볼거면 'all-MiniLM-L6-v2' 같은 기본 모델을 써도 된다.
model = SentenceTransformer(base_model_path)

# 임베딩 모델
word_embedding_model = model._first_module()

# Adaptation 하기 전 Tokenize 결과
test_sentence = '행복주택에서 미소금융이나 햇살론을 농협은행에서 담보대출을'
word_embedding_model.tokenizer(test_sentence)


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
No sentence-transformers model found with name /Users/jk/.cache/torch/sentence_transformers/BM-K_KoSimCSE-roberta-multitask. Creating a new one with MEAN pooling.


{'input_ids': [0, 4202, 12636, 27135, 5658, 14554, 15351, 9778, 2570, 2069, 6470, 4701, 27135, 6484, 2104, 2102, 2069, 2], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [10]:
from transformers import AutoModel, AutoTokenizer

# 토큰화된 결과 확인하기 (Adaptation 적용 전)

# base_model_path = "BM-K/KoSimCSE-roberta-multitask"
model_base = AutoModel.from_pretrained(base_model_path)
tokenizer_base = AutoTokenizer.from_pretrained(base_model_path)

print(tokenizer_base.tokenize(test_sentence))

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


['행복', '##주택', '##에서', '미소', '##금융', '##이나', '햇살', '##론', '##을', '농협', '##은행', '##에서', '담보', '##대', '##출', '##을']


In [12]:
# Vocabulary 불러오기
added_vocab_df = pd.read_json('./tokenizer/vocab.json', orient='index')

added_vocab_list = []    # 여기에 추가할 단어들 담을거
chk_idx = 0    # 로그 찍기용 index

for new_word in added_vocab_df.index.to_list():
    # 로그 찍기 (10번에 한 번씩)
    chk_idx += 1
    if not chk_idx % 10:
        print(f"{chk_idx}번째 중....{new_word}")

    if '▁' in new_word:
        test_word = new_word.replace('▁', '')
    else:
        test_word = new_word
    test_word = test_word.strip()
    # 이미 vocab에 있는 단어 & 1글자인거 빼고 vocab에 추가
    if test_word not in tokenizer_base.vocab.keys() and len(test_word) > 1:
        added_vocab_list.append(new_word.replace('▁', ''))

# 혹시 모르니 백업
import pickle

with open('./tokenizer/added_vocab_list.pickle', 'wb') as fw:
    pickle.dump(added_vocab_list, fw)

# 본격 토큰 추가

# 앞에서 했던 코드 까먹었을까봐 다시 적음
# base_model_path = "../workspace/da_finetune_epoch_2"
# model = SentenceTransformer(base_model_path)
# word_embedding_model = model._first_module()

word_embedding_model.tokenizer.add_tokens(added_vocab_list, special_tokens=False)
word_embedding_model.auto_model.resize_token_embeddings(len(word_embedding_model.tokenizer))


10번째 중....7
20번째 중....H
30번째 중....R
40번째 중....b
50번째 중....l
60번째 중....w
70번째 중....갑
80번째 중....거
90번째 중....결
100번째 중....곳
110번째 중....군
120번째 중....글
130번째 중....께
140번째 중....난
150번째 중....냄
160번째 중....념
170번째 중....뇨
180번째 중....닉
190번째 중....달
200번째 중....덧
210번째 중....되
220번째 중....듀
230번째 중....따
240번째 중....란
250번째 중....량
260번째 중....력
270번째 중....롤
280번째 중....륙
290번째 중....릭
300번째 중....말
310번째 중....먼
320번째 중....모
330번째 중....물
340번째 중....밖
350번째 중....버
360번째 중....벽
370번째 중....봇
380번째 중....붙
390번째 중....빼
400번째 중....상
410번째 중....성
420번째 중....솔
430번째 중....쉼
440번째 중....식
450번째 중....쓰
460번째 중....앓
470번째 중....야
480번째 중....엇
490번째 중....엽
500번째 중....와
510번째 중....운
520번째 중....윗
530번째 중....응
540번째 중....있
550번째 중....쟁
560번째 중....제
570번째 중....죄
580번째 중....직
590번째 중....착
600번째 중....척
610번째 중....촉
620번째 중....충
630번째 중....침
640번째 중....컴
650번째 중....쿠
660번째 중....킴
670번째 중....택
680번째 중....토
690번째 중....트
700번째 중....판
710번째 중....평
720번째 중....풍
730번째 중....하
740번째 중....했
750번째 중....혁
760번째 중....홈
770번째 중....횟
780번째 중.

Embedding(35316, 768)

In [13]:
# Adaptation 하기 전 Tokenize 결과
word_embedding_model.tokenizer(test_sentence)


{'input_ids': [0, 33044, 3604, 34243, 3642, 32345, 1498, 6470, 34102, 6484, 2104, 2102, 2069, 2], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [14]:
# from transformers import AutoModel, AutoTokenizer
# base_model_path = '../workspace/da_finetune_epoch_2'
# model_base = AutoModel.from_pretrained(base_model_path)
# tokenizer_base = AutoTokenizer.from_pretrained(base_model_path)

# 토큰들 추가
print(len(tokenizer_base.vocab))
added_token_num = tokenizer_base.add_tokens(added_vocab_list)
print(len(tokenizer_base.vocab))

# model에도 바뀐거 적용
print(model_base.get_input_embeddings())
model_base.resize_token_embeddings(tokenizer_base.vocab_size + added_token_num)
print(model_base.get_input_embeddings())

# id -> word 바꾸도록 dictionary 생성
vocab_id_2_word = {v: k for k, v in tokenizer_base.vocab.items()}
# dictionary 이용해서 아까 나온 결과 확인
input_ids = word_embedding_model.tokenizer(test_sentence)['input_ids']
print([vocab_id_2_word[t_id] for t_id in input_ids])


32000
35316
Embedding(32000, 768, padding_idx=1)
Embedding(35316, 768)
['[CLS]', '행복주택', '에서', '미소금융', '이나', '햇살론', '을', '농협', '은행에서', '담보', '##대', '##출', '##을', '[SEP]']


In [17]:
save_path = './workspace/epoch_tok'
model_name = 'epoch_tok'
model.save(path=save_path, model_name=model_name)

In [16]:
new_model = SentenceTransformer(save_path)
new_word_embedding_model = new_model._first_module()

input_ids = new_word_embedding_model.tokenizer(test_sentence)['input_ids']

print([vocab_id_2_word[t_id] for t_id in input_ids])

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


['[CLS]', '행복주택', '에서', '미소금융', '이나', '햇살론', '을', '농협', '은행에서', '담보', '##대', '##출', '##을', '[SEP]']


In [30]:
from torch.utils.data import DataLoader
import json
from sentence_transformers import InputExample
from sentence_transformers import losses
from sentence_transformers.evaluation import InformationRetrievalEvaluator

In [23]:
train_dataset_path = './da_train/da_train_dataset.json'
val_dataset_path = './da_train/da_val_dataset.json'

#BATCH_SIZE
batch_size = 8

In [24]:
with open(train_dataset_path, 'r+', encoding='utf-8') as f :
    train_dataset = json.load(f)

with open(val_dataset_path, 'r', encoding='utf-8') as f :
    val_dataset = json.load(f)

In [26]:
corpus = train_dataset['corpus']
queries = train_dataset['queries']
relevant_docs = train_dataset['relevant_docs']

examples = []

for query_id, query in queries.items():
    node_id = relevant_docs[query_id][0]
    text = corpus[node_id]
    example = InputExample(texts=[query, text])
    examples.append(example)

In [27]:
loader = DataLoader(
    examples, batch_size=batch_size,
)

In [29]:
loss = losses.MultipleNegativesRankingLoss(new_model)

In [31]:
# Define evaluator

corpus = val_dataset['corpus']
queries = val_dataset['queries']
relevant_docs = val_dataset['relevant_docs']

evaluator = InformationRetrievalEvaluator(queries, corpus, relevant_docs)

In [32]:
# epoch config
EPOCHS = 3

In [33]:
warmup_steps = int(len(loader) * EPOCHS * 0.1)

#memory allocation error
model.to('mps')

model.fit(
    train_objectives=[(loader, loss)],
    epochs=EPOCHS,
    warmup_steps=warmup_steps,
    output_path='./da_finetune',
    show_progress_bar=True,
    evaluator=evaluator, 
    evaluation_steps=50,
)

Epoch:   0%|          | 0/3 [00:00<?, ?it/s]

Iteration:   0%|          | 0/1177 [00:00<?, ?it/s]

KeyboardInterrupt: 